# Ground Navigation for Aerial Vehicles

With a satellite reference image, we intend to utilize a SfM solution to match ground segments of images to the reference map.
- Inputs: Reference image (satellite), SfM solution (images, points, cameras), selected images (5)
- Output: Correction solutions for translation, rotation, scaling

Checklist
1. [x] Load in satellite image values 
2. [ ] Choose ground segment in each image
3. [ ] Load ground segments into a plot
4. [ ] Implement an initial guess
5. [ ] Perform SSD process on each
6. [ ] Create a correction vector for each
7. [ ] Visualization of each correction vector

In [9]:
import numpy as np
import cv2
import open3d as o3d
import plotly.graph_objects as go
import plotly.io as pio
from scipy.spatial.transform import Rotation as R
from scipy.spatial import cKDTree
import imageio
%matplotlib qt
import matplotlib.pyplot as plt

from groundNAV_agent import *
from colmapParsingUtils import *

# SAVE YOUR WORK
%load_ext autoreload
%autoreload 2
%autosave 180

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [2]:
# NOTES: 
# OS in JUMBOS: IMG_9446; ID = 4
# 40yd (far hash): IMG_9520; ID = 78
# Jumbo logo: IMG_9531; ID = 89
# 30yd (near hash): IMG_9542; ID = 100
# U in JUMBOS: IMG_9576; ID = 134

In [3]:
# Load in necessary parameters for gNAV agent 
# Define Class Parameters 

images_colm  = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/images.txt"
cameras_colm = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/cameras.txt"
pts3d_colm = "/home/daniel-choate/ASAR/s2/TerrainNav/TTurf/test/points3D_f.txt"

# Images selected for local corrections
image_1 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9446.JPEG" #ID:4
image_2 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9520.JPEG" #ID:78
image_3 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9531.JPEG" #ID:89
image_4 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9542.JPEG" #ID:100
image_5 = "/home/daniel-choate/Datasets/COLMAP/TTurfSAT/TTurf_Im/IMG_9576.JPEG" #ID:134
# Load in satellite reference image
sat_ref = "TTurf/TurfSat.jpg"
# sat_ref = cv2.imread('TTurf/TurfSat.jpg')

# Organize for agent params
images = [image_1, image_2, image_3, image_4, image_5]

In [4]:
# Create class
gnav = gNAV_agent(images_colm, cameras_colm, pts3d_colm, images, sat_ref)

# Grab raw points and RGB data for scene and reference cloud
scene_pts, rgb_data = gnav.grab_pts(gnav.pts3d_c)
ref_pts, ref_rgb = gnav.ref_pts, gnav.ref_rgb

In [ ]:
# Use ground plane pts to set reference frame 
# Need gravity and height
pts_gnd_idx = np.array([25440, 25450, 25441, 25449, 25442, 25445, 103922, 103921, 103919, 103920])
tform_ref_frame = gnav.set_ref_frame(pts_gnd_idx)
print("\nReference frame transformation\n", tform_ref_frame)

In [ ]:
# Transform all points to the new coordinate system 
# Not necessary since we aren't using the cloud, but a good visual check for coord frame
tform_ref_frame_inv = gnav.inv_homog_transform(tform_ref_frame)
origin_ref, scene_pts_ref, scene_vec_ref = gnav.unit_vec_tform(scene_pts, gnav.origin_w, tform_ref_frame_inv)
# print(origin_ref)
# Transform scene cloud to 2D (also as a visual check)
# Note: 2d projection will look off with z=-1; see TTurf_v2 for cropping method
scene_ref_2d = gnav.proj_2d_scene(scene_pts_ref)
# print(scene_ref_2d)

### Grab ground plane points from each desired image 

In [ ]:
# Image 1

In [ ]:
# Image 2

In [ ]:
# Image 3

In [ ]:
# Image 4

In [ ]:
# Image 5

In [ ]:
x,y = 150, 1600
side_x = 2500 # WIDTH
side_y = 1000 # HEIGHT

# Plot to visualize
# gnav.plot_rect_im(x, y, side_x, side_y) 

# Get necessary location and rghb data 
pts_loc, pts_rgb = gnav.grab_image_pts(x, y, side_x, side_y)
# print(pts_loc)

# Plotting Tools

In [ ]:
# # PLOTTING THE ORIGINAL SCENE WITH THE ORIGINAL ORIGIN 

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Originial scene with ORIGINAL ORIGIN")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for scene points
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(scene_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # PLOTTING the SATELLITE REFERENCE image as a cloud

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Originial scene with ORIGINAL ORIGIN")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=500)

# # Create point cloud for scene points
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(ref_pts)
# scene_cloud.colors = o3d.utility.Vector3dVector(ref_rgb)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(scene_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # PLOTTING THE ORIGINAL SCENE WITH THE REFERENCE ORIGIN 

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="Originial scene with REFERENCE ORIGIN")

# # Create axes @ origin
# axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1)

# # Create point cloud for scene points
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(scene_pts_ref)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# # Add necessary geometries to visualization 
# vis.add_geometry(axis_origin)
# vis.add_geometry(scene_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()

In [ ]:
# # PLOTTING THE ORIGINAL SCENE WITH THE REFERENCE ORIGIN 

# # Use open3d to create point cloud visualization 
# # Create visualization 
# vis = o3d.visualization.Visualizer()
# vis.create_window(window_name="2D PROJECTION scene with REFERENCE ORIGIN")

# # Create axes @ origin
# # axis_origin = o3d.geometry.TriangleMesh.create_coordinate_frame(size=100)

# # Create point cloud for scene points
# scene_cloud = o3d.geometry.PointCloud()
# scene_cloud.points = o3d.utility.Vector3dVector(scene_ref_2d)
# scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

# # Add necessary geometries to visualization 
# # vis.add_geometry(axis_origin)
# vis.add_geometry(scene_cloud)

# # # Size options (jupyter gives issues when running this multiple times, but it looks better)
# # render_option = vis.get_render_option()
# # render_option.point_size = 2

# # Run and destroy visualization 
# vis.run()
# vis.destroy_window()